<p style="font-size:22pt;font-family:'Ubuntu';">Similar Names Corelations</p>

<br>

This document is to present corelations between polish names based on their similarity. The data comes from external file (in this case xlsx sheet which contains **16417 rows and 4 columns** about ranking of popular polish names) which will be loaded and used in this workbook. The result will be presened in a [Directed graph](https://en.wikipedia.org/wiki/Directed_graph) as a hierarchy of particular similar groups. 

The algorithm is simple - we decide which names are should be grupped together in a hierarchy based on how much cost(insertion, deletion, substitution of particular letters in a word) it takes to get desired similarity. 

Similarity is measured by [Gaussian Kernel algorithm](http://mccormickml.com/2013/08/15/the-gaussian-kernel/) which allows us to perform [Affinity Propagation](https://en.wikipedia.org/wiki/Affinity_propagation) on the data and then create hierarchy groups based on the similarity.


list of modules used in this worksheet includes **openpyxl** to load xlsx file,
**pandas** to work on out file (DataFrame),
**matplotlib** for gaussian kernel graphical demonstration,
**sklearn.cluster** to clusterize our words based on costs,
**graphviz** - to draw result as a directed graph, 
python built in libraries **collections** (default dict) and **itertools** (product, permutations), 
and ultimately of course, **numpy**;

this worksheet was created with help of Paweł Wołoszyn

note: sometimes graphviz has a problem with rendering results inside jupyter. If you come across this kind of trouble, check this project directory, images will be stored there eventually.




In [ ]:
#some lines are hidden due to better readability. To show hidden result remove ';' from the end of the command

In [ ]:
from openpyxl import load_workbook

In [ ]:
workbook = load_workbook('names.xlsx')
workbook.sheetnames;

In [ ]:
worksheet = workbook.active

In [ ]:
""" play with worksheet """;

In [ ]:
worksheet.calculate_dimension()


In [ ]:
list(worksheet.values)[-1]

In [ ]:
list(worksheet.values)[1]

In [ ]:
list(worksheet.values)[0]

In [ ]:
list(worksheet.column_dimensions)

In [ ]:
list(worksheet.values)[10][1]

In [ ]:
import pandas as pd
pd.read_excel('names.xlsx');

In [ ]:
head_row, *rows = worksheet.values
head_row;

In [ ]:
""" load our data into the workhorse """
names_data = pd.DataFrame(rows, columns=head_row) 
names_data;

In [ ]:
names_data.head();

In [ ]:
names_data['Płeć'][3];

In [ ]:
""" return unique values of names_data. Its does not sort values """

uniques = names_data['Imię'].unique();

In [ ]:
""" group data by name column and return total number of occurences"""

grouped_names = names_data[['Imię', 'Liczba']].groupby('Imię')
len(uniques) == len(grouped_names);

In [ ]:
""" ranking of unique gruped names """

grouped_ranking = grouped_names.sum().sort_values(by='Liczba', ascending=False)
grouped_ranking;

In [ ]:
names = grouped_ranking.index[:300].values
test_names = names[:10]
test_names;

In [ ]:
POLISH_ALPHABET = 'AĄBCĆDEĘFGHIJKLŁMNŃOÓPQRSŚTUVWXYZŹŻ'
POLISH_VOWELS = 'AĄEĘIOÓUY'
POLISH_SIMILAR = ['CĆ', 'IJY', 'LŁ', 'NŃ', 'OÓU', 'SŚ', 'VW', 'ZŹŻ']
POLISH_LESS_SIMILAR = ['BP', 'GK', 'FH', 'MN', 'CKX', 'CSZ']
POLISH_PAIRS = ['CH', 'GH', 'PH', 'SH', 
                'CZ', 'DZ', 'RZ', 'SZ',
                'IA', 'IE', 'II', 'IO', 'IU',
                'CI', 'NI', 'SI', 'ZI',
                'JJ', 'KK', 'LL', 'ŁŁ', 'MM', 'NN', 'PP', 'RR', 'SS', 'ZZ',
                'CS', 'KS', 'PS']

In [ ]:
POLISH_LESS_SIMILAR_COST = 0.6
POLISH_SIMILAR_COST = 0.4
POLISH_VOWELS_SUBSTITUTION_COST = 0.2

POLISH_VOWELS_INDEL_COST = 1.5
POLISH_PAIR_COST = 0.5

In [ ]:
from collections import defaultdict
class PairsCosts(defaultdict):
    """ class for making costs of words pairs """
    
    def __init__(self, default_cost=1):
        """ this is pretty much the same as dict build in type, but sets default cost to 1 if not provided """
        super().__init__(lambda: defaultdict(lambda:default_cost))
        
    def set_pairs(self, iterable, cost):
        """ set cost of a pair only in a polish pair group """
        
        for first, second in iterable:
            self[first][second] = cost
            
    def set_reflexive(self, iterable, cost):
        """ set cost of the same letter """
        
        for item in iterable:
            self[item][item] = cost
            
    def set_cliques(self, cliques, cost):
        """ set cost of a pair in groups """
        
        from itertools import permutations
        for clique in cliques:
            for first, second in permutations(clique, 2):
                self[first][second] = cost
     
    def set_product (self, first_iterable, second_iterable, cost):
        """ set costs as a cartesian product of two iterables """
        
        from itertools import product
        for first, second in product(first_iterable, second_iterable):
            self[first][second] = cost
    

In [ ]:
def substitution_costs (alphabet, similarity_groups):
    """ calculate substitution costs for each subgroup """
    
    costs = PairsCosts()
    for subgroups, cost in similarity_groups:
        costs.set_cliques(subgroups, cost)
    costs.set_reflexive(alphabet, 0)
    return costs

In [ ]:
def indel_costs (alphabet, single_groups, pair_groups):
    """ calculate cartesian product for vowels and pair costs for pairs """
    
    costs = PairsCosts()
    for group, cost in single_groups:
        costs.set_product(alphabet, group, cost)
    for group, cost in pair_groups:
        costs.set_pairs(group, cost)
    return costs

In [ ]:
polish_indel_costs = indel_costs(POLISH_ALPHABET, 
                                 [(POLISH_VOWELS, POLISH_VOWELS_INDEL_COST)], 
                                 [(POLISH_PAIRS, POLISH_PAIR_COST)])

In [ ]:
polish_substitution_costs = substitution_costs(POLISH_ALPHABET, 
                                               [(POLISH_SIMILAR, POLISH_SIMILAR_COST), 
                                               (POLISH_LESS_SIMILAR, POLISH_LESS_SIMILAR_COST),
                                               ([POLISH_VOWELS], POLISH_VOWELS_SUBSTITUTION_COST)])

In [ ]:
""" check costs of some groups """

polish_substitution_costs['Z'].items()

In [ ]:
polish_indel_costs['Z'].items()

In [ ]:
import numpy as np

In [ ]:
def calculate_distance(source, target, substitution_costs=None, 
                       indel_costs=None, force_symmetric=True):
    
    """ returns minimum distance as a float point number based on costs to get from source to target """
    
    source_lengths = len(source) + 1 
    target_lengths = len(target) + 1
    
    distances = np.zeros((source_lengths, target_lengths)) # initialize two dimension array full of 0 as a distance
    distances[0, :] = range(target_lengths)
    distances[:, 0] = range(source_lengths)
    
    for s in range(1, source_lengths):
        for t in range(1, target_lengths):
            
            insertion = 1 
            deletion = 1
            
            source_prefix = source[:s]
            target_prefix = target[:t]
            
            if indel_costs is not None:
                deletion = indel_costs[target_prefix[-1]][source_prefix[-1]]
                if t > 1:
                    insertion = indel_costs[target_prefix[-2]][target_prefix[-1]]
                
            if substitution_costs is not None:
                substitution = substitution_costs[source_prefix[-1]][target_prefix[-1]]
            else:
                substitution = 0 if target_prefix[-1] == source_prefix[-1] else 1
    
            distances[s, t] = min(
                distances[s-1, t] + deletion,
                distances[s, t-1] + insertion,
                distances[s-1, t-1] + substitution
            )

    result = distances[-1, -1]
    if force_symmetric:
        reverse_result = calculate_distance(target, source, 
                                       substitution_costs, indel_costs, 
                                       force_symmetric=False)
        result = (result + reverse_result) / 2
    return result
    

In [ ]:
calculate_distance('OLA', 'ANIA');

In [ ]:
calculate_distance('MIECZYSŁAW', 'HUNEGUNDA');

In [ ]:
def random_word(alphabet, minl, maxl):
    from random import randint, choices
    length = randint(minl, maxl)
    return ''.join(choices(alphabet, k=length))


In [ ]:
random_word(POLISH_ALPHABET, 2, 5)

In [ ]:
""" tests """;

In [ ]:
import unittest

class TestEditDistance (unittest.TestCase):
    
    def word (self):
        return random_word(POLISH_ALPHABET, 5, 10)
    def metric (self, first, second):
        return calculate_distance(first, second, polish_substitution_costs, polish_indel_costs)
    
    def test_nonnegativity (self):
        for n in range(100):
            distance = self.metric(self.word(), self.word())
            self.assertGreaterEqual(distance, 0)
            
    def test_symmetry (self):
        for n in range(100):
            x, y = self.word(), self.word()
            distance = self.metric(x, y)
            reverse_distance = self.metric(y, x)
            self.assertEqual(distance, reverse_distance)
            
    def test_identity (self):
        for n in range(100):
            w = self.word()
            distance = self.metric(w, w)
            self.assertEqual(distance, 0)

    def test_triangle_inequality (self):
        for n in range(100):
            x, y, z = self.word(), self.word(), self.word()
            straight_distance = self.metric(x, z)
            through_distance = self.metric(x, y) + self.metric(y, z)
            self.assertGreaterEqual(through_distance, straight_distance)

unittest.main(argv=[''], exit=False, verbosity=2);

In [ ]:
def gaussian_kernel(x, delta=None):
    """ gaussian kernel algorithm to calculate similarity between two words """
    
    return np.exp(-x**2 / (2 * (delta or x.std())**2))

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(0, 10, num=51)
plt.plot(x, gaussian_kernel(x));

In [ ]:
def affinity_matrix (words: np.array, substitution_costs=None, indel_costs=None):
    """ returns similarity between two words as a 0-1 float point number  """
    
    def distance (i, j):
        return calculate_distance(words[i], words[j], substitution_costs, indel_costs)
    vectorized_distance = np.vectorize(distance)
    print(vectorized_distance)
    distances = np.fromfunction(vectorized_distance, words.shape * 2, dtype=int)
    return gaussian_kernel(distances)

In [ ]:
similarity = affinity_matrix(test_names, polish_substitution_costs, polish_indel_costs);

In [ ]:
print(np.array_repr(similarity)); # similarity for each word in test_names, determined by gaussian kernel algorithm

In [ ]:
def clusterize (words: np.array, substitution_costs=None, indel_costs=None):
    """perform affinity propagation clustering on names data """
    
    from sklearn.cluster import AffinityPropagation
    classifier = AffinityPropagation(affinity='precomputed')
    affinities = affinity_matrix(words, substitution_costs, indel_costs)
    clustering = classifier.fit_predict(affinities)
    exemplars = classifier.cluster_centers_indices_
    return clustering, exemplars

In [ ]:
def groups (words, substitution_costs=None, indel_costs=None):
    """ create clusterized groups based on similarity"""
    
    words = np.array(words)
    clusters, exemplars = clusterize(words, substitution_costs, indel_costs)
    def cluster (n):
        exemplar = exemplars[n]
        indices = np.where(clusters == n)[0]
        members = indices[indices != exemplar]
        return words[exemplar], list(words[members])
    return dict(cluster(n) for n in range(len(exemplars)))

In [ ]:
groups(test_names, polish_substitution_costs, polish_indel_costs);

In [ ]:
def hierarchy_groups (words, substitution_costs=None, indel_costs=None):
    """ create hierarchy groups based on clusterized similar groups """
    
    hierarchy = {word: {} for word in words}
    old_len, new_len = None, len(hierarchy)
    while old_len != new_len and new_len > 1:
        words = list(hierarchy.keys())
        grouping = groups(words, substitution_costs, indel_costs)
        for exemplar, members in grouping.items():
            for member in members:
                member_group = hierarchy.pop(member)
                hierarchy[exemplar][member] = member_group
        old_len, new_len = new_len, len(hierarchy)
    return hierarchy

In [ ]:
from graphviz import Graph, Digraph

In [ ]:
def populate (graph, hierarchy):
    """ add nodes and edges to graph """
    
    for key, subdict in hierarchy.items():
        graph.node(key)
        for subkey in subdict:
            graph.edge(key, subkey)
        populate(graph, subdict)

In [ ]:
graph = Digraph(engine='sfdp')
hierarchy = hierarchy_groups(names, polish_substitution_costs, polish_indel_costs)
populate(graph, hierarchy)
graph.attr(overlap='false', splines='true')
graph.render('names', view=False) #display results